# Мониторинг Spark приложений

# Доступные сервисы

1. Демоны Hadoop:
    - демон [NameNode](http://localhost:9870) запущен на `9870` порту,
    - демон [DataNode](http://localhost:9864) запущен на `9864` порту,
    - демон [ResourceManager](http://localhost:8088) запущен на `8088` порту порту,
    - демон [NodeManager](http://localhost:8042) запущен на `8042` порту.
2. Сервис [Prometheus](http://localhost:9090) запущен на `9090` порту,
3. Сервис [Grafana](http://localhost:3000) запущен на `3000` порту,
4. Сервис [Prometheus PushGateway](http://localhost:9091) запущен на `9091` порту.

# Мотивация

Мониторинг приложения состоит из трех частей:

- **логирование**: приложение на всем протяжении своей жизни пишет сообщения о происходящих действиях,
- **трассировка**: важно иметь возможность отследить весь путь запроса назависимо является ли приложение частью большой системы или нет,
- **сбор метрик**: приложение в процессе своей жизни использует какие-то ресурсы, поэтому очень важно следить за доступность ресурсов.

Мониторинг является важным аспектом при разработке программного обеспечения, т.к. он позволяет получить ценные сведения о состоянии, производительности и поведения приложний и систем. Среди основных причин можно выделить:

1. **поиск и решение проблем**: мониторинг позволяет отслеживать производительность приложения в реальном времени. Это позволяет отлавливать ошибки и ограничения производительности. Постоянный мониторинг системы ведет к быстрому обнаружению проблем и принятию необходимых действий по их предупреждению;
2. **оптимизация производительности**: мониторинг позволяет анализировать использование ресурсов, время отклика и другие показатели. Это позволяет находить проблемные зоны приложения, которые необходимо оптимизировать для повышения эффективности. Через мониторинг можно как понять, как система ведет себя в различных сценариях, так и принять взвешенное решение об архитектуре системы для повышения производительности;
3. **планирование ресурсов и масштабирование**: мониторинг предоставляет ценные данные относительно трендов использования ресурсов. Анализируя эти данные, можно предсказать потребности приложения в будущем, а также разработать стратегию масштабирования приложения и ресурсов. Такой подход позволяет быть уверенным, что приложение сможет эффективно справляться в будущем с более высокими нагрузками;
4. **безопасность и надежность**: мониторинг позволяет отлавливать и предотвращать риски связанные с безопасностью. Мониторинг позволяет выделить аномальные паттерны поведения или сомнительные операции для проактивного принятия превентивных мер;
5. **отладка**: подробное логирование позволяет записать события и действия, выполняемые в приложении. Логи позволяют воссоздать контекст проблемы ретроспективно, что дает программистам возможность сузить область поиска проблемы для эффективного ее решения.

Мониторинг позвляет сформировать полное представление о работе приложения, что положительно сказывается на его поддержке, развитии и наждежности.

## Метрики

Apache Spark может предоставить метрики в нескольких форматах:

- HTTP (Prometheus, Graphite),
- JMX,
- CSV.

В современном мире для сбора метрик наиболее часто используется [Prometheus](https://prometheus.io/), а для визуализации используется [Grafana](https://grafana.com/), поэтому примеры будут приводиться на их основе.

Конфигурация метрик лежит в `$SPARK_HOME/conf/metrics.properties`. По умолчанию файл отсутствует или является пустым:

In [ ]:
! ls -l $SPARK_HOME/conf/metrics.properties

Включить поддержку метрик для экспорта в `Prometheus` можно например, таким образом:

In [ ]:
! cat /tmp/spark/metrics-prometheus-servlet.properties

## Логирование

В качестве библиотеки для логирования Apache Spark использует [log4j2](https://logging.apache.org/log4j/2.x/), а файл конфигурации доступен для редактирования, что позволяет настроить логирование в соответствии с любыми требованиями бизнеса. Файл конфигурации логера располагается по пути `$SPARK_HOME/conf/log4j2.properties`. По умолчанию файл отсутствует или пустой:

In [ ]:
! ls -l $SPARK_HOME/conf/log4j2.properties

На примере ниже можно увидеть пример настройки консольного логера от уровня `WARN` и выше (~~`TRACE`~~, ~~`DEBUG`~~, ~~`INFO`~~, `WARN`, `ERROR`, `FATAL`):

In [ ]:
! cat /tmp/spark/log4j2-warn.properties

Подробнее уровни логирования разбираются ниже.

### Mapped Diagnostic Context - MDC

Apache Spark позволяет добавить в логи информацию о метаданных задачи через `MDC` переменную(*) `taskName`. Так пользователь при описании формата вывода данных может использовать это значение как и все остальные `MDC` переменные: `%X{mdc.taskName}`

(*) `MDC` переменные - это локальные переменные потока (Thread Local).

In [ ]:
! grep --color taskName /tmp/spark/log4j2-warn.properties

Традиционно логи приложения отправлялись в [ElasticSearch](https://www.elastic.co) или [OpenSearch](https://opensearch.org/), но все больше приложений выбирают для логов [Loki](https://grafana.com/oss/loki/). Loki - это современный стек для работы с логами от компании, которая развивает Grafana, поэтому примеры работы с логами будут выполняться на основе Loki. При желании, можно использовать другие системы хранения логов: Flume, ElasticSearch, OpenSearch и другие, если сконфигурировать соответствующий логер в файле `$SPARK_HOME/conf/log4j2.properties`.

## Трассировка

Apache Spark генерирует новый java код под каждый запрос, а значит большая часть кода является недоступной для анализа после завершения работы запроса, поэтому трассировка Apache Spark приложений не получила большого распространения. Apache Spark не предоставляет способов трассировки запросов, но можно воспользоваться [OpenTelemetry](https://opentelemetry.io/) для этих целей.

## OpenTelemetry

Стандарт [OpenTelemetry](https://opentelemetry.io/) появился в 2021 году как результат слияния двух стандартов: [OpenTracing](https://opentracing.io/) и [OpenCensus](https://opencensus.io/).

Философия OpenTelemetry заключается в унификации трех аспектов мониторинга: метрики, логи, трейсы под одним стандартом.

Также важной частью OpenTelemetry является концепция **коллектора** (collector), который представляет собой прокси объект для обработки поступающей информации, преобразования ее и передачи ее дальше. Все исходящие из коллектора данные преобразуются к одному формату (**OTLP**, **OpenTelemetry Protocol**) и могут быть направлены для дальнейшей обработки нижележащим системам, способным обрабатывать данные в этом формате.

Для `JVM` приложений, коим является Apache Spark, имеется удобный `javaagent`, который инструментирует код, добавляя необходимую логику в рантайме. Так, приложение, в котором изначально не было возможностей для телеметрии, может получить их, если будет запущенно с OpenTelemetry Java агентом. OpenTelemetry Java агент знает о большом числе технологий (Log4j2, Kafka, Cassandra и т.д.) и знает как их инструментировать, т.е. добавлять новую логику для сбора логов, метрик и трейсов.

### Конфигурация

Cтандартный способ [конфигурации OpenTelemetry Java Agent](https://github.com/open-telemetry/opentelemetry-java/blob/57d83344178f578eb5d2f043b0ffc5c42b6719a5/sdk-extensions/autoconfigure/README.md) заключается в установке значений в:
- переменных окружения `OTEL_*`,
- системных переменных `-Dotel.*`.

OpenTelemetry Java агент не входит в стандартную поставку Apache Spark, поэтому необходимо скачать агента самостоятельно:

In [ ]:
! curl -L -o /tmp/opentelemetry-javaagent.jar \
    https://github.com/open-telemetry/opentelemetry-java-instrumentation/releases/latest/download/opentelemetry-javaagent.jar

После загрузки необходимо указать агента при запуске Apache Spark. Дополнительную конфигурацию для старта `JVM` можно добавить:

- через переменную окружения `PYSPARK_SUBMIT_ARGS`: `export PYSPARK_SUBMIT_ARGS='--driver-java-options "-javaagent:/tmp/opentelemetry-javaagent.jar" ...'`;
- через параметр `spark.driver.defaultJavaOptions` при старте приложения Spark;
- через параметр `spark.driver.extraJavaOptions` при старте приложения Spark.

Разница между `spark.driver.defaultJavaOptions` и `spark.driver.extraJavaOptions` заключается в зоне ответственности:

- параметр `spark.driver.defaultJavaOptions` устанавливается системным администратором в файле `$SPARK_HOME/conf/spark-defaults.conf`,
- параметр `spark.driver.extraJavaOptions` устанавливается программистом во время старта приложения.

По умолчанию файл `$SPARK_HOME/conf/spark-defaults.conf` пуст или отсутствует:

In [ ]:
! ls -l $SPARK_HOME/conf/spark-defaults.conf

Ниже приведен пример конфигурации `spark.driver.extraJavaOptions` через файл конфигурации:

In [ ]:
! cat /tmp/spark/spark-defaults.conf

В конфигурации можно заметить, что OpenTelemetry Java агент не будет экспортировать метрики (`-Dotel.metrics.exporter=none`), т.к. метрики будут экспортироваться стандартными средствами Apache Spark.

Во время запуска конфигурация приложения строится на базе комбинации значений параметров `spark.driver.defaultJavaOptions` и `spark.driver.extraJavaOptions`, причем значения `spark.driver.extraJavaOptions` получают больший приоритет по сравнению с `spark.driver.defaultJavaOptions` при наличии одинаковых системных преременных. Наличие приоритета в применении настроек позволяет программисту переопределить значения, установленные системным администратором.

В окружениях, где воркеры (executor) запускаются отдельным `JVM` процессом (`Yarn`, `k8s`), необоходимо настроить конфигурацию запуска `JVM` при помощи параметров:

- `spark.executor.defaultJavaOptions`,
- `spark.executor.extraJavaOptions`.

Эти параметры аналогичны параметрам `spark.executor.defaultJavaOptions`, `spark.executor.extraJavaOptions`, и за установку каждого параметра отвечает отдельная команда.

Логи отправляются OpenTelemetry коллектору, который запущен в виде отдельного Docker сервиса `collector`:

In [ ]:
! source ~/.bash_aliases && \
docker compose ps collector

Приложение Apache Spark знает о запущенном OpenTelemetry коллекторе через переменную окружения `OTEL_EXPORTER_OTLP_ENDPOINT`:

In [ ]:
import os
print(os.environ['OTEL_EXPORTER_OTLP_ENDPOINT'])

# Запуск Spark с OpenTelemetry Java Агентом

## Конфигурация приложения

Приложение Spark необходимо запустить с OpenTelemetry Java агентом. Конфигурация java агента является кандидатом на использование `spark.driver.defaultJavaOptions`, значение которой устанавливает системный администратор в файле `$SPARK_HOME/conf/spark-defaults.conf`. В данный момент этот файл недоступен для редактирования, поэтому необходимо воссоздать начальную конфигурацию приложения Spark в другом месте.

Скопируем все конфигурационные файлы на новое место:

In [ ]:
! rm -rf /tmp/spark-conf && \
cp -v -R $SPARK_HOME/conf /tmp/spark-conf

Заменим содержимое `spark-defaults.conf` файла:

In [ ]:
! cp /tmp/spark/spark-defaults.conf /tmp/spark-conf/spark-defaults.conf && \
cat /tmp/spark-conf/spark-defaults.conf

Заменим содержимое `log4j2.properties` файла:

In [ ]:
! cp /tmp/spark/log4j2-warn.properties /tmp/spark-conf/log4j2.properties && \
cat /tmp/spark-conf/log4j2.properties

Заменим содержимое `metrics.properties` файла:

In [ ]:
! cp /tmp/spark/metrics-prometheus-servlet.properties /tmp/spark-conf/metrics.properties && \
cat /tmp/spark-conf/metrics.properties

Apache Spark должен знать, что актуальная конфигурация находится теперь располагается в `/tmp/spark-conf`, поэтому необходимо установить переменную окружения `SPARK_CONF_DIR`. Переменная окружения `SPARK_CONF_DIR` указывает Apache Spark на место хранения актуальной конфигурации:

In [ ]:
import os
os.environ['SPARK_CONF_DIR'] = '/tmp/spark-conf'

Теперь все приложения, запускаемые на этой машине, будут использовать конфигурацию из директории, на которую указывает `SPARK_CONF_DIR` (`/tmp/spark-conf`)

## Запуск приложения

In [ ]:
from pyspark.sql import SparkSession

Особенность запуска Apache Spark приложения в текущем ноутбуке заключается в том, что метрики и логи должны иметь одинаковое значение имени приложения/сервиса для удобного анализа приложения через Grafana:

In [ ]:
service_name = "MonitoringLocal"

Метрики получают имя приложения через параметр `spark.app.name` (задается через [SparkSession.builder#appName](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.SparkSession.builder.appName.html)), а логи получают имя сервиса через системную переменную `-Dotel.service.name`, поэтому очень важно задать одинаковые значения для `spark.app.name` и `-Dotel.service.name`:

In [ ]:
spark = (
    SparkSession
        .builder
        .appName(service_name)
        .master("local[4]")
        .config("spark.driver.extraJavaOptions", f"-Dotel.service.name={service_name}")
        .config("spark.executor.extraJavaOptions", "${spark.driver.extraJavaOptions}")
        .config("spark.ui.prometheus.enabled", True)
        .config("spark.executor.processTreeMetrics.enabled", True)
        .config("spark.metrics.namespace", "${spark.app.name}")
        .getOrCreate()
)
sc = spark.sparkContext

Для демонстрации нагрузки запустим несколько задач на кластере:

In [ ]:
for i in range(100):
    spark.sql(f"select '{i}.Hello, Monitoring!' as message").collect()

## Логирование

### Текущая конфигурация

Текущее Spark приложение сохраняет логи в Loki при помощи OpenTelemetry Java агента.

Все логи сначала отправляются в Open Telemetry коллектор, который развернут в Docker сервисе `collector`:

In [ ]:
! source ~/.bash_aliases && \
docker compose ps collector

А OpenTelemetry коллектор отправляет логи в Loki, который развернут в Docker сервисе loki:

In [ ]:
! source ~/.bash_aliases && \
docker compose ps loki

### Конфигурация коллектора

Коллектор выступает прокси сервером между приложением Spark и Loki. Конфигурация коллектора выглядит следующим образом:

In [ ]:
! source ~/.bash_aliases && HOST=dind execute \
cat conf/otel/config.yaml

Конфигурационный файл OpenTelemetry коллектора разделен на три секции:

- `receivers` (источники) - какие форматы данных текущий коллектор готов принимать,
- `exporters` (приемники) - в какие системы текущий коллектор готов отправлять данные,
- `service` (логика) - описывает навигацию (ETL) данных между источниками и приемниками через pipeline. Число pipeline объектов в конфигурационном файле может быть больше одного.

Подробно узнать про возможности коллектора и особенности его конфигурации можно в [документации](https://opentelemetry.io/docs/collector/configuration/).

### Создание логера

Чтобы воспользоваться возможностями инструментированного логера, необходимо получить объект логера текущего Spark приложения из `JVM`. Функция `get_logger` позволяет получить ссылку на java-логер:

In [ ]:
def get_logger(logger_name: str):
    return spark._jvm.org.apache.log4j.LogManager.getLogger(logger_name)

Функция `get_logger` принимает _**произвольное**_ текстовое значение в качестве параметра, которое будет использоваться в как имя логгера:

### Использование Логеров

Для создания логера необходимо вызывать функцию `get_logger`. Результирующий объект имеет методы для логирования на разном уровне:

- `trace`,
- `debug`,
- `info`,
- `warn`,
- `error`,
- `fatal`.

In [ ]:
logger_hello_world = get_logger("Hello World")
logger_hello_world.error("Log a ERROR level message")

Логи становятся доступны в трех местах:

- стандартный поток вывода PySpark,
- стандартный поток вывода OpenTelemetry Collector,
- хранилище Loki.

Cтандартный поток вывода PySpark:

In [ ]:
! source ~/.bash_aliases && \
docker compose logs pyspark --tail 100 | grep 'Log a ERROR level message'

Стандартный поток вывода OpenTelemetry Collector:

In [ ]:
! source ~/.bash_aliases && \
docker compose logs collector --tail 100 | grep 'Log a ERROR level message'

Хранилище Loki:

In [ ]:
! curl -G -s "http://loki:3100/loki/api/v1/query_range" \
  --data-urlencode 'query={job="MonitoringLocal", level="ERROR"}' \
  --data-urlencode 'limit=1' | json_pp | grep body | sed 's,^\s\+",,;s,"$,,;s,\\",",g' | json_pp | grep '"Log a ERROR level message"'

Можно залогировать сообщения с уровнем `WARN`:

In [ ]:
logger_hello_world = get_logger("Hello World")
logger_hello_world.warn("Log a WARN level message")

In [ ]:
! curl -G -s "http://loki:3100/loki/api/v1/query_range" \
  --data-urlencode 'query={job="MonitoringLocal", level="WARN"}' \
  --data-urlencode 'limit=1' | json_pp | grep body | sed 's,^\s\+",,;s,"$,,;s,\\",",g' | json_pp | grep '"Log a WARN level message"'

Можно залогировать сообщения с уровнем `INFO`:

In [ ]:
logger_hello_world = get_logger("Hello World")
logger_hello_world.info("Log an INFO level message")

In [ ]:
! curl -G -s "http://loki:3100/loki/api/v1/query_range" \
  --data-urlencode 'query={job="MonitoringLocal", level="INFO"}' \
  --data-urlencode 'limit=1' | json_pp | grep body | sed 's,^\s\+",,;s,"$,,;s,\\",",g' | json_pp |& grep '"Log a INFO level message"' 2> /dev/null

### Уровни логирования

В реальном мире приложение может сообщать о разных событиях, каждое из которых может быть важным на разных этапах жизненного цикла приложения. Так, например, разработчикам нужна подробная информация о приложении во время разработки. Во время эксплуатации могут интересовать только ошибки и предупрежения. Библиотека `Log4j2` имеет шесть уровней логирования:

1. `TRACE`,
1. `DEBUG`,
1. `INFO`,
1. `WARN`,
1. `ERROR`,
1. `FATAL`.

Уровни логирования имеют приоритет. Уровни логирования в списке выше перечислены от минимального приоритета к максимальному.

Каждое сообщение в логе имеет свой уровень логирования. Во время запуска приложения можно указать требуемый уровень логирования. В логах появляются как сообщения, относящиеся указанному уровню логирования, так и все сообщения, относящиеся к уровням более высокого приоритета. Например, если при старте приложения указать уровень `INFO`, то в логах появятся сообщения, как относящиеся к `INFO`, так и к `WARN, ERROR, FATAL`.

В дополнение к перечисленным выше уровням логирования `Log4j2` предлагает еще два уровня:

- `ALL` показать все сообщения (аналогично `TRACE`),
- `OFF` отключить логирование полностью.

Как было отмечено выше, конфигурация логера выполняется через `$SPARK_HOME/conf/log4j2.properties` файл, **НО** текущее приложение запущено с конфигурацией в директории `$SPARK_CONF_DIR`, поэтому файл конфигурации логера находится в `$SPARK_CONF_DIR/log4j2.properties`.

При отсутствии файлов `$SPARK_CONF_DIR/log4j2.properties` или `$SPARK_HOME/conf/log4j2.properties`:

- значение уровня логера по умолчанию будет `WARN`;
- файл `$SPARK_HOME/conf/log4j2.properties` можно создать самостоятельно. В качестве примера можно использовать `$SPARK_HOME/conf/log4j2.properties.template`.

Первая строчка в файле `$SPARK_HOME/conf/log4j2.properties` указывает уровень логирования. В данном случае указан уровень `WARN` (`rootLogger.level = WARN`). Поэтому логах появляются только сообщения, относящиеся к `WARN`, `ERROR`, `FATAL`.

In [ ]:
! cat $SPARK_CONF_DIR/log4j2.properties

Через конфиг `spark.log.level` Apache Spark позволяет переопределить уровень логирования, с которого сообщения необходимо выводить в лог. Если не указать никакой уровень, то значение по умолчанию - `WARN`.

Таким образом, Spark выводит в лог сообщения, начиная с уровня `coalesce(spark.log.level, rootLogger.level, WARN)`, где `coalesce` - функция, которая возвращает первое непустое значение среди своих аргументов.

### Изменение уровня логирования

Уровень логирования можно задать как при старте приложения, так и изменить уровень логирования запущенного приложения.

У запущенного приложения простое изменения значения параметра `spark.log.level` не сработает:

In [ ]:
import sys
try:
    spark.conf.set("spark.log.level", "INFO")
except Exception as e:
    print(e, file=sys.stderr)

Изменение уровня логирования необходимо выполнить через [`SparkContext#setLogLevel`](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.SparkContext.setLogLevel.html). Функция `update_log_level` абстрагирует изменение уровня логирования:

In [ ]:
def update_log_level(log_level: str):
    spark.sparkContext.setLogLevel(log_level)

Функция `update_log_level` принимает уровень логирования в качестве параметра. Например, чтобы изменить уровень логирования на `INFO`, необходимо запустить:

In [ ]:
update_log_level("INFO")

После изменения уровня логирования, можно увидеть, что сообщения уровня `INFO` появляются в логах:

In [ ]:
logger_hello_world = get_logger("Hello World")
logger_hello_world.info("Log an INFO level message. Once Again")

In [ ]:
! curl -G -s "http://loki:3100/loki/api/v1/query_range" \
  --data-urlencode 'query={job="MonitoringLocal"}' \
  --data-urlencode 'limit=1' | json_pp | grep body | sed 's,^\s\+",,;s,"$,,;s,\\",",g' | json_pp | grep '"Log an INFO level message. Once Again"'

In [ ]:
spark.stop()

### Вывод

Apache Spark использует библиотеку `log4j2`, что позволяет следовать принятым в индустрии практикам логирования сообщений. Разделение логируемых сообщений на уровни облегчает эксплуатацию приложений, т.к. можно эффективно ограничивать поток логируемых сообщений в зависимости от фазы жизненного цикла приложения. Гибкость `log4j2` позволяет реализовать требования к конфигурации любого рода, а использование OpenTelemetry Java агента позволяет без усилий подключить любые системы для сбора логов. В то же время необходимо учитывать, что логирование несколько снижает производительность приложения, поэтому очень важно выбирать наиболее подходящий к ситуации уровень логирования.

### Задания

Для кода ниже ответье на вопросы
```python
logger = get_logger("Н. Н. Некрасов. Крестьянские дети")

logger.trace("Однажды, в студеную зимнюю пору,")
logger.debug("Я из лесу вышел; был сильный мороз.")
logger.info("Гляжу, поднимается медленно в гору")
logger.warn("Лошадка, везущая хворосту воз.")
logger.error("И, шествуя важно, в спокойствии чинном,")
logger.fatal("Лошадку ведет под уздцы мужичок.")
```

1. Приложение запущено с настройками по умолчанию. Какие из сообщений ниже попадут в логи? Проверьте на практике.
1. Настроить приложение так, чтобы были залогированы все сообщения.
1. Настроить приложение так, чтобы ниодного сообщения не было залогировано.
1. Настроить приложение так, чтобы было залогировано только одно (любое) сообщение.

<details>
    <summary>Нужна помощь?</summary>

    Ответы внизу страницы
</details>

## Метрики

### Конфигурация

Для активации экспорта метрик в Prometheus необходимо установить два параметра при старте приложения:

- `spark.executor.processTreeMetrics.enabled` - нужно ли собирать метрики с виртуальной файловой системы `/proc`;
- `spark.ui.prometheus.enabled` - делает доступными метрики воркеров на эндпоинте `/metrics/executors/prometheus`.

Виртуальная файловая система `/proc` представляет информацию о текущем состоянии операционной системы, процессах, памяти и т.д. Например, можно получить информацию о процессорах, установленных на текущей машине:

In [ ]:
! grep 'processor' -A 5 /proc/cpuinfo | head -n 14

Метрики приложения настраиваются при помощи файлов `$SPARK_CONF_DIR/metrics.properties` (если `SPARK_CONF_DIR` установлено) или `$SPARK_HOME/conf/metrics.properties`:

In [ ]:
! cat $SPARK_CONF_DIR/metrics.properties

На основании содержимого файла `$SPARK_CONF_DIR/metrics.properties` можно заключить, что метрики будут доступны в формате [Prometheus](https://prometheus.io/).

Если файл `$SPARK_CONF_DIR/metrics.properties` или `$SPARK_HOME/conf/metrics.properties` отсутствует, то можно создать `$SPARK_CONF_DIR/metrics.properties` (если `SPARK_CONF_DIR` установлена) или `$SPARK_HOME/conf/metrics.properties` самостоятельно. Для вдохновения можно обратить внимание на `$SPARK_HOME/conf/metrics.properties.template`:

In [ ]:
! tail $SPARK_HOME/conf/metrics.properties.template

Если выбрана конфигурация метрик для Prometheus, то вместе с приложением Spark стартует Prometheus сервлет рядом с **Spark Web UI** (порт [`4040`](http://localhost:4040/metrics/prometheus) по умолчанию). Prometheus должен забирать метрики с двух эндпоинтов:

- [`/metrics/prometheus`](http://localhost:4040/metrics/prometheus) - метрики драйвера,
- [`/metrics/executors/prometheus`](http://localhost:4040/metrics/executors/prometheus) - метрики воркеров.

### Запуск Yarn приложения

Приложение Spark будет запущено в Yarn, поэтому необходимо обеспечить Docker сервис `hadoop`, на котором будут запущены Yarn контейнеры воркеров OpenTelemetry Java агентом:

In [ ]:
! source ~/.bash_aliases && HOST=hadoop execute \
curl -L -o /tmp/opentelemetry-javaagent.jar \
    https://github.com/open-telemetry/opentelemetry-java-instrumentation/releases/latest/download/opentelemetry-javaagent.jar

In [ ]:
import os

service_name = "MonitoringYarn"

spark = (
    SparkSession
        .builder
        .appName(service_name)
        .master("yarn")
        .config("spark.driver.extraJavaOptions", f"-Dotel.service.name={service_name}")
        .config("spark.executor.extraJavaOptions", f"${{spark.driver.extraJavaOptions}} -Dotel.exporter.otlp.endpoint={os.environ['OTEL_EXPORTER_OTLP_ENDPOINT']}")
        .config("spark.ui.prometheus.enabled", True)
        .config("spark.executor.processTreeMetrics.enabled", True)
        .config("spark.metrics.namespace", "${spark.app.name}")
        .config("spark.log.level", "ALL")
        .getOrCreate()
)
sc = spark.sparkContext

Запустим несколько задач для симуляции рабочей нагрузки:

In [ ]:
for i in range(100):
    spark.sql(f"select '{i}.Hello, Yarn Monitoring!' as message").collect()

### Метрики драйвера

При анализе метрик драйвера (управляющая программа) можно заметить интересную особенность:

In [ ]:
! curl -s -L http://localhost:4040/metrics/prometheus | head

Кажая метрика начинается с префикса `metrics_MonitoringYarn`. Если обратить внимание на конфигурацию запуска приложения, то можно увидеть, что `MonitoringYarn` - это значение `spark.app.name`:

In [ ]:
print(spark.conf.get("spark.app.name"))

А также значение переменной `service_name`, которое было передано в [SparkSession.builder#appName](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.SparkSession.builder.appName.html):

In [ ]:
print(service_name)

Но свой префикс метрики получают не от имени приложения, а через еще один параметр `spark.metrics.namespace`:

In [ ]:
print(spark.conf.get("spark.metrics.namespace"))

Apache Spark допускает ссылки на значения других параметров, поэтому `spark.metrics.namespace` становится равным `MonitoringYarn` (значение `spark.app.name`).

Таким образом, **Apache Spark** выставляет метрики, названия которых зависят от значения параметра `spark.metrics.namespace`.

> **Имена метрик зависят от `spark.metrics.namespace`!**

Но почему нельзя было использовать значение `spark.metrics.namespace` в виде метки (**label**) метрики?

<details>
    <summary>Гипотезы</summary>

1. Более точная настройка метрик;
1. В качестве меток (**label**) желательно использовать низкокардинальные признаки (число уникальных значений мало), а всевозможных значений `spark.metrics.namespace` может быть бесконечно.
</details>

Значения по умолчанию для `spark.metrics.namespace` нет, поэтому если запустить приложение без указания значения для `spark.metrics.namespace`, то Apache Spark будет использовать идентификатор приложения для генерации префикса. Таким образом префикс будет выглядит следующим образом:

In [ ]:
print(f'metrics_{spark.conf.get("spark.app.id")}')

Идентификатор приложения меняется от запуска к запуску, поэтому при перезапуке приложения нельзя будет объединить значения старых метрик в Prometheus с новыми.

> **Необходимо всегда определять значение `spark.metrics.namespace`!**

### Метрики воркеров

Метрики воркеров также доступны с хоста драйвера, что делает удобным сбор метрик:

In [ ]:
! curl -s -L http://localhost:4040/metrics/executors/prometheus | head

В отличии от метрик драйвера, метрики воркеров связаны с приложением через метки (**label**):

- `application_id`,
- `application_name`.

Такой подход позволяет использовать универсальные [`PromQL`](https://prometheus.io/docs/prometheus/latest/querying/basics/) (язык запросов Prometheus) запросы для получения результатов.

In [ ]:
spark.stop()

### Системы сбора и визуализации метрик

Prometheus (сбор метрик) и Grafana (визуализация метрик) запущены в виде Docker сервисов и доступны через веб интерфейс:

- сервис [Prometheus](http://localhost:9090) запущен на порту `9090`. Логин и пароль не требуются;
- сервис [Grafana](http://localhost:3000) запущен на порту `3000`. Логин: `admin` , пароль: `admin`

### Генерация нагрузки

Для нагурзки будет использоваться алгоритм [PageRank](https://en.wikipedia.org/wiki/PageRank) на синтетическом графе.

#### Генерация синтетического графа

Генерация синтетического графа заключается в генерации рёбер графа:

In [ ]:
! for i in {1..500}; do \
    for j in {1..100}; do \
        echo "$i $j"; \
    done ; \
done > /tmp/edges.log

In [ ]:
! head /tmp/edges.log

Полученный файл необходимо отрпавить в Docker сервис `hadoop`, с которого можно будет загрузить файл в **HDFS**:

In [ ]:
! source ~/.bash_aliases && \
docker compose cp pyspark:/tmp/edges.log /tmp/edges.log && \
docker compose cp /tmp/edges.log hadoop:/tmp/edges.log

Отправка файла в **HDFS**:

In [ ]:
! source ~/.bash_aliases && \
hdfs dfs -mkdir -p /user/jovyan && \
hdfs dfs -put -f /tmp/edges.log /user/jovyan/edges.log

Проверка наличия файла в **HDFS**:

In [ ]:
! source ~/.bash_aliases && \
hdfs dfs -ls /user/jovyan/edges.log

#### Конфигурация нагрузки

Нагрузка будет выполняться для приложения запущенного в _клиентском_ режиме на Yarn. Следовательно, воркеры будут запущены в виде Yarn контейнеров, логи которых также представляют ценность. Значит необходимо запустить воркеры с OpenTelemetry Java агентом для отправки логов в OpenTelemetry коллектор.

Конфигурация процессов JVM для контейнеров осуществляется при помощи параметров:

- `spark.executor.defaultJavaOptions` - параметры, устанавливаемые системным администратором в `$SPARK_CONF_DIR/spark-defaults.conf` или `$SPARK_HOME/conf/spark-defaults.conf`,
- `spark.executor.extraJavaOptions` - параметры, устанавливаемые программистом.

In [ ]:
! cat $SPARK_CONF_DIR/spark-defaults.conf

Параметр `spark.log.level` не оказывает влияния на воркеры, которые запущены в виде отдельных JVM процессов, воркеры используют конфигурацию из файла `log4j2.properties`. Для воркеров будет установлен уровень `INFO`:

In [ ]:
! sed -i '/WARN/s,WARN,INFO,' $SPARK_CONF_DIR/log4j2.properties && \
cat $SPARK_CONF_DIR/log4j2.properties

#### Запуск нагрузки

_(Во время тестирования приложение ниже запускалось только после перезапуска hadoop, поэтому перезапускаю его превентивно)_

In [ ]:
! source ~/.bash_aliases && \
docker compose restart hadoop

In [ ]:
! source ~/.bash_aliases && \
docker compose ps hadoop

Приложение будет запущено в Yarn на двух воркерах в _клиентском_ режиме (`--deploy-mode client`):

In [ ]:
! SPARK_CONF_DIR='/tmp/spark-conf' \
/usr/local/spark/bin/spark-submit --class org.apache.spark.examples.SparkPageRank \
    --master yarn \
    --deploy-mode client \
    --num-executors 2 \
    --conf spark.driver.extraJavaOptions="-Dotel.service.name=SparkPageRank" \
    --conf spark.executor.extraJavaOptions="\${spark.driver.extraJavaOptions} -Dotel.exporter.otlp.endpoint=${OTEL_EXPORTER_OTLP_ENDPOINT}" \
    --conf spark.log.level=INFO \
    --conf spark.ui.prometheus.enabled=true \
    --conf spark.executor.processTreeMetrics.enabled=true \
    --conf spark.metrics.namespace='${spark.app.name}' \
    /usr/local/spark/examples/jars/spark-examples*.jar \
    /user/jovyan/edges.log 50 > /dev/null

Для анализа результатов необходимо открыть Grafana дашборд под названием [`Spark Applicaitons`](http://localhost:3000). Имя пользователя: `admin` и пароль: `admin`

### Пакетная нагрузка - Batch Processing

Пример выше был умышленно запущен в клиентском режиме. Клиентский режим имеет следующие особенности:

- драйвер (управляющая программа) запущен на машине внешней по отношению к Yarn кластеру,
- драйвер отделен от Application Master,
- Application Master запущен в Yarn контейнере,
- Application Master управляет контейнерами воркеров (executors).

Запуск Spark в клиентском режиме типичен для целей аналитики, при этом стоит принять во внимание:

- драйвер живет неограниченно долго,
- IP адрес и доменное имя машины, на которой запущен драйвер, заранее известно.

Учитывая, что Prometheus использует pull-модель получения метрик, т.е. Prometheus сам обращается к машинам для получения метрик, запуск Apache Spark в клиентском режиме отлично подходит для Prometheus.

Другим режимом работы Apache Spark является кластерный режим:

- драйвер запускается в Yarn контейнере,
- драйвер и Application Master объединены вместе,
- Application Master управляет контейнерами воркеров (executors).

Следующее приложение будет запущено в Yarn на двух воркерах в _кластерном_ режиме (`--deploy-mode cluster`):

In [ ]:
! SPARK_CONF_DIR='/tmp/spark-conf' \
/usr/local/spark/bin/spark-submit --class org.apache.spark.examples.SparkPageRank \
    --master yarn \
    --deploy-mode cluster \
    --num-executors 2 \
    --conf spark.driver.extraJavaOptions="-Dotel.service.name=SparkPageRank" \
    --conf spark.executor.extraJavaOptions="\${spark.driver.extraJavaOptions} -Dotel.exporter.otlp.endpoint=${OTEL_EXPORTER_OTLP_ENDPOINT}" \
    --conf spark.log.level=INFO \
    --conf spark.ui.prometheus.enabled=true \
    --conf spark.executor.processTreeMetrics.enabled=true \
    --conf spark.metrics.namespace='${spark.app.name}' \
    /usr/local/spark/examples/jars/spark-examples*.jar \
    /user/jovyan/edges.log 50 > /dev/null

Драйвер запущен в виде Yarn контейнера, поэтому метрики теперь доступны по адресу: `http://localhost:8088/proxy/<YARN_APP_ID>/metrics/prometheus`.

Кластерный режим работы практикуется при пакетной обработке (batch processing):

- появились новые данные,
- Apache AirFlow, Apache Oozie или любой другой планировщик запустил Spark приложение,
- Spark приложение обработало новые данные,
- Spark приложение отключилось.

Небольшой вариацией вышеописанного сценария является стримовая обработка (streaming), в котором:

- драйвер запущен постоянно,
- размещение Yarn контейнера для драйвера определяется во время запуска,
- расположение Yarn контейнера для драйвера неизвестно заранее.

Особенностью кластерного режима можно считать:

- неопределенное время жизни драйвера,
- ни IP адрес, ни доменное имя машины драйвера неизвестно, т.к. yarn выделяет контейнеры для работы по ресурсам.

Следовательно, для Spark приложений, запущенных в кластерном режиме, невозможно сконфигурировать Prometheus корректно, т.к. нет конкретного домена, который выставляет метрики.

> **Кластерный режим Apache Spark не подходит для Prometheus!**

### Prometheus PushGateway

Логичным решением сбора метрик для Spark приложений в кластерном режиме должен быть push метод:

- приложения стартуют в любой момент,
- приложения разворачиваются в любом месте,
- приложения самостоятельно в комфортном для себя режиме отправляют собранные метрики на некоторый центральный сервер.

Команда Prometheus прекрасно осведомлена о проблемах со сбором метрик с приложений, которые не имеют ни фиксированного времени жизни, ни доменного имени. Чтобы не терять долю рынка, которая занята пакетной обработкой, был разработан [Prometheus PushGateway](https://prometheus.io/docs/practices/pushing/):

- Prometheus PushGateway представляет собой веб-сервер,
- команда приложения (Spark) разворачивает Prometheus PushGateway сервер в своем окружении,
- у Prometheus PushGateway сервера появляется фиксированное доменное имя,
- приложение (Spark) отправлет метрики на домен сервера Prometheus PushGateway,
- Prometheus PushGateway выставляет полученные метрики в виде эндпонта,
- Prometheus забирает метрики, выставленные Prometheus PushGateway сервером.

Таким образом, Prometheus PushGateway выступает прокси-сервером между приложением (Spark) без фиксированного домена и Prometheus.

Для решения проблемы сбора метрик с приложений Spark, запущенных в кластерном режиме, в текущем окружении поднят Prometheus PushGateway сервис в Docker сервисе `pushgateway`:

In [ ]:
! source ~/.bash_aliases && \
docker compose ps pushgateway

Полученные метрики `pushgateway` доступны по адресу:

In [ ]:
! curl -s -k http://pushgateway:9091/metrics | head -n 5

### Конфигурация Spark для Prometheus PushGateway

Apache Spark не поддерживает Prometheus PushGateway по умолчанию, но компания BanzaiCloud любезно [открыла](https://github.com/banzaicloud/spark-metrics) доступ к своему модулю Apache Spark для Prometheus PushGateway.

Для использования Prometheus PushGateway необхдимо выполнить дополнительную конфигурацию Spark:

1. положить jar файл на `CLASSPATH` драйвера (управляющая программа) и воркеров (executors),
2. добавить конфигурацию в `metrics.properties`.

При старте текущего Docker compose сервисов необходимый `jar` файл попадает в директорию `$SPARK_HOME/jars`:

In [ ]:
! ls -l $SPARK_HOME/jars/spark-metrics-assembly-3.2-1.0.0.jar

Файлы `*.jar` из `$SPARK_HOME/jars` формируют `CLASSPATH` приложения. Все jar файлы из `$SPARK_HOME/jars` автоматически копируются на все контейнеры воркеров и драйвера (если приложение запущено в кластерном режиме).

Конфигурация для Prometheus PushGateway выполняется согласно [документации](https://github.com/banzaicloud/spark-metrics/blob/1f2147fac5103a34e68bbc74a9021a00d3b5ad37/PrometheusSink.md) в файле `$SPARK_CONF_DIF/metrics.properties` или `$SPARK_HOME/conf/metrics.properties`.

Минимальная конфигураци включает в себя следующие пункты:

- активация PrometheusSink,
- указание домена Prometheus PushGateway.

In [ ]:
! cp /tmp/spark/metrics-prometheus-pushgateway.properties /tmp/spark-conf/metrics.properties && \
cat /tmp/spark-conf/metrics.properties

Теперь можно запускать Apache Spark приложения в кластерном режиме, и все метрики будут отправляться в Prometheus PushGateway.

### Запуск Spark с Prometheus PushGateway

Все ноутбуки запускаются исключительно в клиентском режиме, поэтому для запуска приложения в кластерном режиме необходимо использовать `spark-submit`.

Для демонстрации будет запущен тот же самый PageRank пример в кластерном режиме (`--deploy-mode cluster`):

In [ ]:
! SPARK_CONF_DIR='/tmp/spark-conf' \
$SPARK_HOME/bin/spark-submit --class org.apache.spark.examples.SparkPageRank \
    --master yarn \
    --deploy-mode cluster \
    --num-executors 2 \
    --conf spark.driver.extraJavaOptions="-Dotel.service.name=SparkPageRank" \
    --conf spark.executor.extraJavaOptions="\${spark.driver.extraJavaOptions} -Dotel.exporter.otlp.endpoint=${OTEL_EXPORTER_OTLP_ENDPOINT}" \
    --conf spark.log.level=INFO \
    --conf spark.ui.prometheus.enabled=true \
    --conf spark.executor.processTreeMetrics.enabled=true \
    --conf spark.metrics.namespace='${spark.app.name}' \
    $SPARK_HOME/examples/jars/spark-examples*.jar \
    /user/jovyan/edges.log 5 > /dev/null

Для проверки доступности метрик как с драйвера, так и с воркеров, можно открыть PushGateway на порту [9091](http://localhost:9091) и убедиться в наличии метрик.

## Вывод

Все приложения нуждаются в оперативном реагировании на инциденты, а мониторинг позволяет их предупредить. Несмотря на достаточно слабые встроенные возможности мониторинга Spark приложений, современные технологии в лице OpenTelemetry протягивают руку помощи, что позволяет настроить информативный мониторинг. В свою очередь OpenTelemetry добавляет накладные расходы, что может снизить производительность Spark приложений, но за эту цену можно получить надежность приложения. В свою очередь программист также может повлиять на производительность приложений Spark, если хорошо понимает, какие возможности Apache Spark предлагает для настройки логирования и мониторинга.

## Задания

1. Запустить `org.apache.spark.examples.SparkPi` из `$SPARK_HOME/examples/jars/spark-examples*.jar` при помощи `spark-submit`:
    - в кластерном режиме,
    - c уровнем логирования `OFF`,
    - мониторингом на базе `Prometheus PushGateway`,
    - проверить метрики.
1. Запустить `org.apache.spark.examples.SparkPi` из `$SPARK_HOME/examples/jars/spark-examples*.jar` при помощи `spark-submit`:
    - в клиентском режиме,
    - c уровнём логирования `DEBUG`,
    - c мониторингом на базе `PrometheusServlet`,
    - проверить метрики.
1. Запустить Spark приложение:
    - через ноутбук, 
    - в локальном режиме,
    - с уровнем логирования `ERROR`,
    - c мониторингом на базе `Prometheus PushGateway`

<details>
    <summary>Нужна помощь?</summary>

    Ответы внизу страницы.
</details>

## Метрики Apache Hadoop

Apache Hadoop выставляет свои метрики по эндпоинту `/prom`:

- метрики [NameNode](http://localhost:9870/prom),
- метрики [DataNode](http://localhost:9864/prom),
- метрики [ResourceManager](http://localhost:8088/prom),
- метрики [NodeManager](http://localhost:8042/prom).

Apache Hadoop выставляет свои метрики, только если параметр `hadoop.prometheus.endpoint.enabled` в `core-site.xml` установлен в `true`:

In [ ]:
! source ~/.bash_aliases && HOST=dind execute \
cat conf/hadoop/core-site.xml

# Ответы

## Ответы на задание про логирование

Для кода ниже ответье на вопросы
```python
logger = get_logger("Н. Н. Некрасов. Крестьянские дети")

logger.trace("Однажды, в студеную зимнюю пору,")
logger.debug("Я из лесу вышел; был сильный мороз.")
logger.info("Гляжу, поднимается медленно в гору")
logger.warn("Лошадка, везущая хворосту воз.")
logger.error("И, шествуя важно, в спокойствии чинном,")
logger.fatal("Лошадку ведет под уздцы мужичок.")
```

1. Приложение запущено с настройками по умолчанию. Какие из сообщений ниже попадут в логи? Проверьте на практике.

<details>
    <summary>Ответ</summary>

1. По умолчанию:
    - настройка `spark.log.level` отсутствует,
    - файл `$SPARK_HOME/conf/log4j2.properties` отсутствует
1. Итоговый уровень логгирования: `coalesce(spark.log.level (null), rootLogger.level (null), WARN) = WARN`;
1. В результате будут залогированы сообщения уровней: `WARN`, `ERROR`, `FATAL`.

**ОТВЕТ**: в лог попадут сообщения:
```python
logger.warn("Лошадка, везущая хворосту воз.")
logger.error("И, шествуя важно, в спокойствии чинном,")
logger.fatal("Лошадку ведет под уздцы мужичок.")
```

</details>

2. Настроить приложение так, чтобы были залогированы все сообщения.

<details>
    <summary>Ответ</summary>

1. Остановить текущее приложение: `spark.stop()`
2. Запустить приложение с `spark.log.level=ALL`:
```python
spark = (
    SparkSession
        .builder
        .appName(service_name)
        .master("local[4]")
        .config("spark.driver.extraJavaOptions", f"-Dotel.service.name={service_name}")
        .config("spark.log.level", "ALL")
        .getOrCreate()
)
```
3. залогировать сообщения для проверки
```python
logger = get_logger("Н. Н. Некрасов. Крестьянские дети")

logger.trace("Однажды, в студеную зимнюю пору,")
logger.debug("Я из лесу вышел; был сильный мороз.")
logger.info("Гляжу, поднимается медленно в гору")
logger.warn("Лошадка, везущая хворосту воз.")
logger.error("И, шествуя важно, в спокойствии чинном,")
logger.fatal("Лошадку ведет под уздцы мужичок.")
```

4. Остановить приложение: `spark.stop()`

</details>

3. Настроить приложение так, чтобы ниодного сообщения не было залогировано.

<details>
    <summary>Ответ</summary>

1. Остановить текущее приложение: `spark.stop()`
2. Запустить приложение с `spark.log.level=OFF`:
```python
spark = (
    SparkSession
        .builder
        .appName(service_name)
        .master("local[4]")
        .config("spark.driver.extraJavaOptions", f"-Dotel.service.name={service_name}")
        .config("spark.log.level", "OFF")
        .getOrCreate()
)
```
3. залогировать сообщения для проверки
```python
logger = get_logger("Н. Н. Некрасов. Крестьянские дети")

logger.trace("Однажды, в студеную зимнюю пору,")
logger.debug("Я из лесу вышел; был сильный мороз.")
logger.info("Гляжу, поднимается медленно в гору")
logger.warn("Лошадка, везущая хворосту воз.")
logger.error("И, шествуя важно, в спокойствии чинном,")
logger.fatal("Лошадку ведет под уздцы мужичок.")
```

4. Остановить приложение: `spark.stop()`

</details>

4. Настроить приложение так, чтобы было залогировано только одно (любое) сообщение.

<details>
    <summary>Ответ</summary>

    Идея в том, что нужно указать `spark.log.level=FATAL` при старте прилжения. Так будет залогировано единственное сообщение с этим уровнем.

1. Остановить текущее приложение: `spark.stop()`
2. Запустить приложение с `spark.log.level=FATAL`:
```python
spark = (
    SparkSession}
        .builder
        .appName(service_name)
        .master("local[4]")
        .config("spark.driver.extraJavaOptions", f"-Dotel.service.name={service_name}")
        .config("spark.log.level", "FATAL")
        .getOrCreate()
)
```
3. залогировать сообщения для проверки
```python
logger = get_logger("Н. Н. Некрасов. Крестьянские дети")

logger.trace("Однажды, в студеную зимнюю пору,")
logger.debug("Я из лесу вышел; был сильный мороз.")
logger.info("Гляжу, поднимается медленно в гору")
logger.warn("Лошадка, везущая хворосту воз.")
logger.error("И, шествуя важно, в спокойствии чинном,")
logger.fatal("Лошадку ведет под уздцы мужичок.")
```

4. Остановить приложение: `spark.stop()`

</details>

## Ответы на задание про запуск приложений

1. Запустить `org.apache.spark.examples.SparkPi` из `$SPARK_HOME/examples/jars/spark-examples*.jar` при помощи `spark-submit`:
    - в кластерном режиме,
    - c уровнем логирования `OFF`,
    - мониторингом на базе `Prometheus PushGateway`,
    - проверить метрики.

<details>
    <summary>Ответ</summary>

1. Настроить Prometheus:

```bash
! cp /tmp/spark/metrics-prometheus-pushgateway.properties /tmp/spark-conf/metrics.properties && \
cat /tmp/spark-conf/metrics.properties
```

2. Настроить логер (один из двух способов):

    - через внешний редактор отредактировать в проекте `spark-exercises` файл `conf/spark/override/log4j2-warn.properties` и скопировать его: `cp /tmp/spark/log4j2-warn.properties /tmp/spark-conf/log4j.properties`
    - через `sed`:

```bash
! sed -i '/rootLogger.level/s,.*,rootLogger.level=OFF,' /tmp/spark-conf/log4j2.properties && \
cat /tmp/spark-conf/log4j2.properties
```

3. запустить приложение:
```bash
! SPARK_CONF_DIR='/tmp/spark-conf' \
$SPARK_HOME/bin/spark-submit --class org.apache.spark.examples.SparkPi \
    --master yarn \
    --deploy-mode cluster \
    --num-executors 2 \
    --conf spark.driver.extraJavaOptions="-Dotel.service.name=SparkPi" \
    --conf spark.executor.extraJavaOptions="\${spark.driver.extraJavaOptions} -Dotel.exporter.otlp.endpoint=${OTEL_EXPORTER_OTLP_ENDPOINT}" \
    --conf spark.log.level=OFF \
    $SPARK_HOME/examples/jars/spark-examples*.jar 10
```

Метрики будут доступны на PushGateway на порту [9091](http://localhost:9091)

</details>

2. Запустить `org.apache.spark.examples.SparkPi` из `$SPARK_HOME/examples/jars/spark-examples*.jar` при помощи `spark-submit`:
    - в клиентском режиме,
    - c уровнём логирования `DEBUG`,
    - c мониторингом на базе `PrometheusServlet`,
    - проверить метрики.

<details>
    <summary>Ответ</summary>

1. Настроить Prometheus:

```bash
! cp /tmp/spark/metrics-prometheus-servlet.properties /tmp/spark-conf/metrics.properties && \
cat /tmp/spark-conf/metrics.properties
```

2. Настроить логер (один из двух способов):

    - через внешний редактор отредактировать в проекте `spark-exercises` файл `conf/spark/override/log4j2-warn.properties` и скопировать его: `cp /tmp/spark/log4j2-warn.properties /tmp/spark-conf/log4j.properties`
    - через `sed`:

```bash
! sed -i '/rootLogger.level/s,.*,rootLogger.level=DEBUG,' /tmp/spark-conf/log4j2.properties && \
cat /tmp/spark-conf/log4j2.properties
```

3. запустить приложение:
```bash
! SPARK_CONF_DIR='/tmp/spark-conf' \
$SPARK_HOME/bin/spark-submit --class org.apache.spark.examples.SparkPi \
    --master yarn \
    --deploy-mode cluster \
    --num-executors 2 \
    --conf spark.driver.extraJavaOptions="-Dotel.service.name=SparkPi" \
    --conf spark.executor.extraJavaOptions="\${spark.driver.extraJavaOptions} -Dotel.exporter.otlp.endpoint=${OTEL_EXPORTER_OTLP_ENDPOINT}" \
    --conf spark.log.level=OFF \
    $SPARK_HOME/examples/jars/spark-examples*.jar 10
```

Метрики будут доступны на дашборде Spark Application в Grafana на порту [3000](http://localhost:9091)

</details>

3. Запустить Spark приложение:
    - через ноутбук, 
    - в локальном режиме,
    - с уровнем логирования `ERROR`,
    - c мониторингом на базе `Prometheus PushGateway`

<details>
    <summary>Ответ</summary>

1. Настроить Prometheus:

```bash
! cp /tmp/spark/metrics-prometheus-pushgateway.properties /tmp/spark-conf/metrics.properties && \
cat /tmp/spark-conf/metrics.properties
```

2. Запустить приложение (редактировать log4j2.properties не нужно, т.к. можно переопределить уровень логирования при помощи `spark.log.level`:

```python
spark = (
    SparkSession
        .builder
        .appName("HomeWork")
        .master("local[4]")
        .config("spark.log.level", "ERROR")
        .config("spark.driver.extraJavaOptions", f"-Dotel.service.name={service_name}")
        .getOrCreate()
)
```

3. Запустить нагрузку:

```python
for i in range(100):
    spark.sql(f"select '{i}.Hello, Monitoring!' as message").collect()
```

Метрики будут доступны на PushGateway на порту [9091](http://localhost:9091)

</details>